In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import shap
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report,
    roc_curve,
    precision_recall_curve,
    ConfusionMatrixDisplay
)

C:\Users\ismai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load data
train_df = pd.read_csv("Tables/bnpl_train.csv")
test_df = pd.read_csv("Tables/bnpl_test.csv")
val_df = pd.read_csv("Tables/bnpl_val.csv")
target_col = "default_flag"

In [3]:
# Quick data checks
print(train_df.shape, test_df.shape, val_df.shape)
print(train_df.head(), test_df.head(), val_df.head())
print("Train nulls in target:", train_df["default_flag"].isna().sum())
print("Train class distribution:\n", train_df["default_flag"].value_counts(normalize=True))

(595, 14) (199, 14) (199, 14)
   external_repayment_loans  credit_card_interest_incidence  default_flag  \
0                         1                               0             0   
1                         0                               0             0   
2                         0                               0             0   
3                         0                               1             0   
4                         1                               1             0   

   bnpl_usage_frequency  financial_stress_score  credit_limit_utilisation  \
0             -0.730258               -1.246089                 -1.746734   
1              1.319109                0.529472                 -0.222965   
2              0.408279                1.239697                  0.805578   
3             -1.641088                1.594809                 -0.184871   
4              0.408279                0.529472                 -1.556263   

   payment_delinquency_count  impulsive_buyi

In [ ]:
# Separate features and target for training set
X_train = train_df.drop(columns=[target_col])
y_train = train_df[target_col]
print("X_train shape:", X_train.shape, X_train.head())
print("y_train shape:", y_train.shape, y_train.head())

X_train shape: (595, 13)    external_repayment_loans  credit_card_interest_incidence  \
0                         1                               0   
1                         0                               0   
2                         0                               0   
3                         0                               1   
4                         1                               1   

   bnpl_usage_frequency  financial_stress_score  credit_limit_utilisation  \
0             -0.730258               -1.246089                 -1.746734   
1              1.319109                0.529472                 -0.222965   
2              0.408279                1.239697                  0.805578   
3             -1.641088                1.594809                 -0.184871   
4              0.408279                0.529472                 -1.556263   

   payment_delinquency_count  impulsive_buying_score  \
0                  -1.500097               -0.528311   
1                  -1

In [ ]:
# Separate features and target for test set
X_test = test_df.drop(columns=[target_col])
y_test = test_df[target_col]
print("X_test shape:", X_test.shape, X_test.head())
print("y_test shape:", y_test.shape, y_test.head())

X_test shape: (199, 13)    external_repayment_loans  credit_card_interest_incidence  \
0                         0                               0   
1                         0                               1   
2                         1                               0   
3                         0                               0   
4                         0                               0   

   bnpl_usage_frequency  financial_stress_score  credit_limit_utilisation  \
0              0.180572               -0.890977                  0.005600   
1             -1.413380                1.594809                  1.643651   
2              1.546817                0.174360                 -1.670546   
3             -1.413380               -0.180752                 -1.327698   
4             -0.047135                0.529472                  1.567463   

   payment_delinquency_count  impulsive_buying_score  \
0                   1.462180                0.504892   
1                  -0.

In [ ]:
# Separate features and target for validation set
X_val = val_df.drop(columns=[target_col])
y_val = val_df[target_col]
print("X_val shape:", X_val.shape, X_val.head())
print("y_val shape:", y_val.shape, y_val.head())

X_val shape: (199, 13)    external_repayment_loans  credit_card_interest_incidence  \
0                         0                               1   
1                         0                               0   
2                         0                               0   
3                         1                               0   
4                         0                               1   

   bnpl_usage_frequency  financial_stress_score  credit_limit_utilisation  \
0             -0.502550                1.239697                  0.272259   
1              0.408279               -0.535864                 -1.670546   
2              0.635987               -0.890977                  0.043694   
3             -0.730258                0.884585                 -0.108683   
4              0.635987                0.529472                 -0.337248   

   payment_delinquency_count  impulsive_buying_score  \
0                  -0.907642                0.160491   
1                   1.4

Model

In [ ]:
# Initialize Random Forest Classifier with basic parameters
rf = RandomForestClassifier(
    n_estimators=100,      
    max_depth=None,        
    random_state=42,       
    n_jobs=-1              
)
# Fit the model
rf.fit(X_train, y_train)

In [ ]:
# Predictions on validation set
y_valid_pred = rf.predict(X_val)
y_valid_proba = rf.predict_proba(X_val)[:, 1]

In [ ]:
#get the best threshold by maximizing F1 score on validation set
thresholds = np.arange(0.0, 1.01, 0.01)
best_threshold = 0
best_f1 = 0

for threshold in thresholds:
    y_valid_pred_thresh = (y_valid_proba >= threshold).astype(int)
    f1 = f1_score(y_val, y_valid_pred_thresh)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold
# Print the best threshold and F1 score
print(f"Best Threshold: {best_threshold}")
print(f"Best F1 Score: {best_f1}")

Best Threshold: 0.37
Best F1 Score: 1.0


In [13]:
y_test_proba = rf.predict_proba(X_test)[:, 1]
y_test_pred = (y_test_proba >= best_threshold).astype(int)

In [ ]:
# Evaluate and print metrics on test set
print(f"--- Metrics at best threshold for f1 score = {best_threshold} ---\n")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))
print("ROC AUC Score:   ", roc_auc_score(y_test, y_test_proba))
print("F1 Score:        ", f1_score(y_test, y_test_pred))
print("Recall Score:    ", recall_score(y_test, y_test_pred))
print("Precision Score: ", precision_score(y_test, y_test_pred))

--- Metrics at best threshold for f1 score = 0.37 ---

Confusion Matrix:
 [[183   0]
 [  3  13]]
ROC AUC Score:    0.98650956284153
F1 Score:         0.896551724137931
Recall Score:     0.8125
Precision Score:  1.0


In [15]:
#get the best features that explain default
feature_names = X_train.columns
importances = rf.feature_importances_
feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)
feature_importance.head(6).to_csv("Results/rf_feature_importance.csv", index=False)
print(feature_importance.head(6))

                      Feature  Importance
11            bnpl_debt_ratio    0.166664
2        bnpl_usage_frequency    0.152238
12   stress_usage_interaction    0.143242
3      financial_stress_score    0.137060
5   payment_delinquency_count    0.120387
4    credit_limit_utilisation    0.095528


In [ ]:
# save results to json
fpr, tpr, thresholds = roc_curve(y_test, y_test_proba)

results = {
    "threshold": best_threshold,
    "confusion_matrix": confusion_matrix(y_test, y_test_pred).tolist(),  # convert to list for saving
    "accuracy": accuracy_score(y_test, y_test_pred),
    "precision": precision_score(y_test, y_test_pred),
    "recall": recall_score(y_test, y_test_pred),
    "f1_score": f1_score(y_test, y_test_pred),
    "roc_auc": roc_auc_score(y_test, y_test_proba),
    "fpr": fpr.tolist(),
    "tpr": tpr.tolist(),
    "roc_thresholds": thresholds.tolist(),
}
with open("Results/RF_model_results.json", "w") as f:
    json.dump(results, f)